# BYOL (Bring Your Own Library) Script Mode

このノートブックでは、独自のライブラリを使用したSageMakerスクリプトモードを体験します。

## 🎯 学習目標
- カスタムライブラリの作成と使用
- Dockerコンテナを使ったBYOL
- 独自アルゴリズムの実装と実行

## ⏱️ 実行時間目安
- **環境準備**: 5-10分
- **カスタムライブラリ作成**: 5分
- **トレーニング実行**: 10-15分
- **合計**: 約25-35分

## 1. 環境設定

In [ ]:
import sagemaker
import boto3
import pandas as pd
import numpy as np
import os
import subprocess
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

# SageMaker設定
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = boto3.Session().region_name
bucket = sagemaker_session.default_bucket()
account_id = boto3.client('sts').get_caller_identity()['Account']

print(f"SageMaker role: {role}")
print(f"Region: {region}")
print(f"S3 bucket: {bucket}")
print(f"Account ID: {account_id}")
print(f"Current directory: {os.getcwd()}")
print("\n🎓 BYOL Script Mode講義を開始します")

## 2. カスタムライブラリの確認

In [ ]:
# BYOLディレクトリの確認
byol_dir = './byol_docker'
if os.path.exists(byol_dir):
    print(f"✅ BYOLディレクトリが存在します: {byol_dir}")
    
    # ファイル一覧表示
    print("\nBYOLファイル構成:")
    for root, dirs, files in os.walk(byol_dir):
        level = root.replace(byol_dir, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            print(f"{subindent}{file}")
else:
    print(f"❌ BYOLディレクトリが見つかりません: {byol_dir}")
    print("\n💡 解決方法:")
    print("1. 前のセルでBYOLファイルが正しく作成されているか確認")
    print("2. ディレクトリパスが正しいか確認")

## 3. カスタムライブラリのテスト（ローカル）

In [ ]:
# カスタムライブラリをローカルでテスト
import sys
sys.path.append('./byol_docker/custom_ml_lib')

try:
    from custom_ml_lib.custom_classifier import CustomEnsembleClassifier
    print("✅ カスタムライブラリのインポート成功")
    
    # 簡単なテストデータで動作確認（パラメータ修正）
    from sklearn.datasets import make_classification
    X_test, y_test = make_classification(
        n_samples=200,           # サンプル数を増加
        n_features=10,           # 特徴量数
        n_informative=8,         # 情報のある特徴量数を増加（重要！）
        n_redundant=2,           # 冗長な特徴量数
        n_classes=3,             # クラス数
        n_clusters_per_class=1,  # クラスあたりのクラスター数を1に
        random_state=42,
        class_sep=1.0            # クラス間の分離度を追加
    )
    
    print(f"テストデータ生成: {X_test.shape}, クラス数: {len(set(y_test))}")
    
    # カスタム分類器のテスト
    classifier = CustomEnsembleClassifier(
        use_rf=True,
        use_gb=True,
        use_lr=True
    )
    
    classifier.fit(X_test, y_test)
    predictions = classifier.predict(X_test[:10])
    probabilities = classifier.predict_proba(X_test[:10])
    
    print(f"予測結果: {predictions}")
    print(f"予測確率形状: {probabilities.shape}")
    print("✅ カスタムライブラリの動作確認完了")
    
except ImportError as e:
    print(f"❌ カスタムライブラリのインポートに失敗: {e}")
except Exception as e:
    print(f"❌ カスタムライブラリのテストに失敗: {e}")

## 4. データ準備

In [ ]:
# データファイルのパス検出
def find_data_path():
    possible_paths = [
        './data/',
        '../data/',
        '~/sagemaker-lecture-version/data/',
        os.path.expanduser('~/sagemaker-lecture-version/data/')
    ]
    
    for path in possible_paths:
        expanded_path = os.path.expanduser(path)
        train_file = os.path.join(expanded_path, 'train_lecture.csv')
        if os.path.exists(train_file):
            return expanded_path
    return None

data_path = find_data_path()

if data_path:
    print(f"✅ データファイル発見: {data_path}")
    
    # データをS3にアップロード
    train_s3_path = sagemaker_session.upload_data(
        os.path.join(data_path, 'train_lecture.csv'),
        bucket=bucket,
        key_prefix='byol-lecture/train'
    )
    
    test_s3_path = sagemaker_session.upload_data(
        os.path.join(data_path, 'test_lecture.csv'),
        bucket=bucket,
        key_prefix='byol-lecture/test'
    )
    
    validation_s3_path = sagemaker_session.upload_data(
        os.path.join(data_path, 'validation_lecture.csv'),
        bucket=bucket,
        key_prefix='byol-lecture/validation'
    )
    
    print(f"✅ データアップロード完了:")
    print(f"  Train: {train_s3_path}")
    print(f"  Test: {test_s3_path}")
    print(f"  Validation: {validation_s3_path}")
else:
    print("❌ データファイルが見つかりません")

## 5. Dockerイメージの構築（オプション）

In [ ]:
# Dockerイメージ構築（時間がかかるため、オプション）
build_docker = False  # Trueに変更してDockerイメージを構築

if build_docker and os.path.exists('./byol_docker/Dockerfile'):
    print("Dockerイメージを構築中...（数分かかります）")
    
    # ECRリポジトリ名
    repository_name = 'sagemaker-byol-lecture'
    image_tag = 'latest'
    
    # ECRのフルURI
    ecr_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{repository_name}:{image_tag}"
    
    try:
        # Dockerイメージ構築
        build_cmd = f"cd ./byol_docker && docker build -t {repository_name}:{image_tag} ."
        result = subprocess.run(build_cmd, shell=True, capture_output=True, text=True)
        
        if result.returncode == 0:
            print("✅ Dockerイメージ構築完了")
            print(f"イメージ名: {repository_name}:{image_tag}")
            
            # ECRにプッシュ（実際の環境では必要）
            print(f"\n💡 ECRにプッシュする場合:")
            print(f"1. aws ecr create-repository --repository-name {repository_name}")
            print(f"2. docker tag {repository_name}:{image_tag} {ecr_uri}")
            print(f"3. docker push {ecr_uri}")
            
        else:
            print(f"❌ Dockerイメージ構築に失敗: {result.stderr}")
            
    except Exception as e:
        print(f"❌ Docker構築エラー: {e}")
else:
    print("⏭️ Dockerイメージ構築をスキップ（build_docker=False）")
    print("\n💡 実際の環境では:")
    print("1. Dockerイメージを構築")
    print("2. ECRにプッシュ")
    print("3. カスタムコンテナでトレーニング実行")

## 6. SageMaker Script Mode（カスタムライブラリ使用）

In [ ]:
# SageMaker Script Modeでカスタムライブラリを使用
# 注意: この例では、カスタムライブラリをsource_dirに含める方法を使用

if 'train_s3_path' in locals():
    from sagemaker.sklearn.estimator import SKLearn
    
    # SKLearnエスティメーター（カスタムライブラリ付き）
    sklearn_estimator = SKLearn(
        entry_point='train.py',
        source_dir='./byol_docker',  # カスタムライブラリを含むディレクトリ
        role=role,
        instance_type='ml.m5.large',
        instance_count=1,
        framework_version='1.0-1',
        py_version='py3',
        hyperparameters={
            'use-custom-ensemble': 'true',
            'ensemble-rf': 'true',
            'ensemble-gb': 'true',
            'ensemble-lr': 'true',
            'feature-selection-k': 20
        },
        output_path=f's3://{bucket}/byol-lecture/output'
    )
    
    print("✅ BYOL SKLearnエスティメーター作成完了")
    print(f"エントリーポイント: train.py")
    print(f"ソースディレクトリ: ./byol_docker")
    print(f"ハイパーパラメータ: {sklearn_estimator.hyperparameters()}")
else:
    print("❌ データがアップロードされていないため、エスティメーター作成をスキップ")

## 7. トレーニング実行

In [ ]:
# BYOLトレーニング実行
if 'sklearn_estimator' in locals():
    print("BYOL Script Modeトレーニング開始...")
    
    import time
    start_time = time.time()
    
    try:
        # トレーニング実行
        sklearn_estimator.fit({
            'train': train_s3_path,
            'test': test_s3_path,
            'validation': validation_s3_path
        }, wait=True)
        
        training_time = time.time() - start_time
        
        print(f"✅ BYOLトレーニング完了: {training_time:.2f}秒")
        print(f"モデル出力: {sklearn_estimator.model_data}")
        
        # トレーニングジョブ名を保存
        training_job_name = sklearn_estimator.latest_training_job.name
        print(f"トレーニングジョブ名: {training_job_name}")
        
    except Exception as e:
        print(f"❌ BYOLトレーニングエラー: {e}")
        print("\n💡 トラブルシューティング:")
        print("1. カスタムライブラリのインポートエラーがないか確認")
        print("2. train.pyのパスが正しいか確認")
        print("3. 依存関係が正しくインストールされているか確認")
else:
    print("❌ エスティメーターが作成されていないため、トレーニングをスキップ")

## 8. 結果の確認

In [ ]:
# トレーニング結果の確認
if 'training_job_name' in locals():
    print("=== BYOL Script Mode 実行結果 ===")
    
    # CloudWatchログの確認（最後の数行）
    try:
        logs_client = boto3.client('logs', region_name=region)
        log_group = f'/aws/sagemaker/TrainingJobs'
        log_stream = training_job_name
        
        response = logs_client.get_log_events(
            logGroupName=log_group,
            logStreamName=log_stream,
            startFromHead=False,
            limit=20
        )
        
        print("\n📋 トレーニングログ（最後の20行）:")
        for event in response['events']:
            print(event['message'].strip())
            
    except Exception as e:
        print(f"ログの取得に失敗: {e}")
    
    print(f"\n📊 実行サマリー:")
    print(f"  トレーニングジョブ: {training_job_name}")
    print(f"  実行時間: {training_time:.2f}秒")
    print(f"  モデル出力: {sklearn_estimator.model_data}")
    print(f"  使用したライブラリ: カスタムアンサンブル分類器")
else:
    print("❌ トレーニングが完了していないため、結果確認をスキップ")

## 9. まとめ

In [ ]:
print("=== BYOL Script Mode 講義完了 ===")
print("\n🎓 学習内容:")
print("✅ カスタムライブラリの作成方法")
print("✅ 独自アルゴリズムの実装")
print("✅ SageMaker Script Modeでのカスタムライブラリ使用")
print("✅ BYOL（Bring Your Own Library）の概念")
print("\n🚀 BYOLの利点:")
print("📦 独自のアルゴリズムを使用可能")
print("🔧 既存のライブラリを活用")
print("⚡ 特定の問題に最適化されたソリューション")
print("🔄 既存のコードベースの再利用")
print("\n🎯 次のステップ:")
print("1. より複雑なカスタムアルゴリズムの実装")
print("2. Dockerコンテナを使った完全なBYOL")
print("3. カスタムライブラリのパッケージ化")
print("4. 本番環境でのデプロイ")
print("\n💡 実際の使用例:")
print("- 企業独自のアルゴリズム")
print("- 研究機関の最新手法")
print("- 特定ドメインに特化したモデル")
print("- レガシーシステムとの統合")